In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import pandas as pd 
from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

In [ ]:
import plotly.graph_objects as go
import plotly

In [ ]:
data_raw = pd.read_csv('E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [ ]:
country_list = data_raw['Country/Region'].unique()
date = data_raw.columns[4:]

In [ ]:
df_SIR = pd.DataFrame({'Date': date})

In [ ]:
for each in country_list:
    df_SIR[each] = np.array(data_raw[data_raw['Country/Region'] == each].iloc[:,4::].sum(axis=0)).T
df_SIR.to_csv("E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/SIR_raw.csv", sep = ';', index=False)

# SIR Calculation

In [ ]:
df_analyse=pd.read_csv('E:/ads_covid-19/data/raw/COVID-19/csse_covid_19_data/SIR_raw.csv',sep=';')  
df_analyse.sort_values('Date',ascending=True)

N0 = 1000000 # Maximum susceptible population
beta = 0.4   # Infection spread dynamics
gamma = 0.1  # Recovery rate

I0=df_analyse.Germany[35] # Starting from 35th day
S0=N0-I0
R0=0

In [ ]:
df_data = df_analyse[35:] # We will consider data from 35th day, which is 26th Feb 2020 
t = np.arange(df_data.shape[0])

In [ ]:
def SIR_model_t(SIR, t, beta, gamma):
    ''' Simple SIR model
        S: Susceptible population
        I: Infected people
        R: Recovered people
        beta: Infection spread dynamics
        gamma: Recovery rate
        
        overall condition is that the sum of changes(differnces) sum up to 0
        dS+dI+dR=0
        
        S+I+R= N (constant size of population)
    
    '''
    S,I,R=SIR
    dS_dt = -beta*I*S/N0
    dI_dt = beta*I*S/N0 - gamma*I
    dR_dt = gamma*I
    
    return dS_dt, dI_dt, dR_dt

In [ ]:
def fit_odeint(x, beta, gamma):
    return integrate.odeint(SIR_model_t, (S0, I0, R0), x, args=(beta, gamma))[:,1] 

In [ ]:
for country in df_data.columns[1:]:
        ydata = np.array(df_data[df_data[country]>0][country]) 
        t = np.arange(len(ydata))
        I0=ydata[0]
        S0=N0-I0
        R0=0
        popt=[0.4,0.1]
        fit_odeint(t, *popt)
        popt, pcov = optimize.curve_fit(fit_odeint, t, ydata, maxfev=5000)
        perr = np.sqrt(np.diag(pcov))
    
        #print('standard deviation errors : ',str(perr), ' start infect:',ydata[0])
        #print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])
        fitted=fit_odeint(t, *popt)
        f_padded = np.concatenate((np.zeros(df_data.shape[0]-len(fitted)) ,fitted)) #to make dimentions equal
        df_data[country + '_fitted'] = f_padded
df_data.to_csv("E:/ads_covid-19/data/processed/SIR_calculated.csv", sep = ';', index=False)        

# SIR Visualization for One Country

In [ ]:
df_data = pd.read_csv('E:/ads_covid-19/data/processed/SIR_calculated.csv',sep=';')
df_data = df_data.reset_index(drop=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df_data.index, 
                             y = df_data['Germany_fitted'],
                             name= 'fitted',
                             mode='markers+lines',
                             line_width = 1,
                             marker_size = 3
                                 
                            ),
                 )

fig.add_trace(go.Scatter(x = df_data.index, 
                             y = df_data['Germany'],
                             name= 'source',
                             mode='markers+lines',
                             line_width = 1,
                             marker_size = 3
                                 
                            ),
                 )

fig.update_layout(title={
                        'text': "Germany",
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                  xaxis_title='Days', 
                  yaxis_title='Population infected',
                  width = 1000, 
                  height = 800)
fig.update_yaxes(type = 'log')
fig.update_layout(xaxis_rangeslider_visible=True)

# SIR Dashboard

- To visualize actual and simulated number of infected population

In [ ]:
import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

In [ ]:
import os
print(os.getcwd())

In [ ]:
df_dash = pd.read_csv('E:/ads_covid-19/data/processed/SIR_calculated.csv',sep=';')
df_dash = df_data.reset_index(drop=True)

In [ ]:
fig = go.Figure()
app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Actual and Simulated Number of Infected People   
    '''),
    
    dcc.Markdown('''
    ##  Plot shows actual number of infected people and simulated number of infected people for different countries derived from SIR model.   
    '''),
    
    dcc.Markdown('''
    ### Select the country:
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_dash.columns[1:187]],
        value=['Germany'], # Which is pre-selected
        multi=True
    ),


    dcc.Graph(figure=fig, id='SIR')
])



@app.callback(
    Output('SIR', 'figure'),
    [Input('country_drop_down', 'value')])
def update_figure(country_list):


    traces = []
    for each in country_list:

        df_plot=df_dash[[each, each+'_fitted']]
        
        traces.append(dict(x=df_plot.index,
                                y=df_plot[each],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )
        traces.append(dict(x=df_plot.index,
                                y=df_plot[each+'_fitted'],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each+'_simulated'
                        )
                )
        
    return {
            'data': traces,
            'layout': dict (
                width=1700,
                height=870,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=20,color="#7f7f7f"), 
                        'titlefont': dict(size=25)
                       
                      },

                yaxis={'type':"log", 'title':'Number of infected people (log-scale)',
                       'tickfont':dict(size=18,color="#7f7f7f"),
                       'titlefont': dict(size=25)
                      },
                
        ) 
    }

if __name__ == '__main__':

    app.run_server(debug=True, use_reloader=False)
